In [56]:
import numpy as np
from scipy.sparse import identity
from scipy import sparse

In [4]:
L = 8

In [21]:
gates_list = open('gates_list_'+str(L)+'.txt','r')

In [22]:
MCX_transpile = []
for gates in gates_list:
    MCX_transpile.append(gates)

### Generating random number vector

In [40]:
number_of_gates = len(MCX_transpile)
Seed = 4000
np.random.seed(Seed)
Noise = np.random.rand(number_of_gates)

### Pre-defined gates

In [47]:
def H_fixed(): # Hadamad gate acting on one qubit.
    
    return 1/np.sqrt(2)*np.matrix([[1,1],[1,-1]])

def Ry(theta):
    return np.matrix([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

def sigma_Z():
    return np.matrix([[1,0],[0,-1]])

def sigma_X():
    return np.matrix([[0,1],[1,0]])

def Rz(theta):
    return np.matrix([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

### Gates with noise

In [59]:
'''
For noise = 0; the following returns the fixed Hadamard gate.
'''
def Hadamard_variable(noise):
    return Ry(np.pi/2+noise)*sigma_Z()

def sigma_X_variable(noise):
    return Hadamard_variable(noise)*sigma_Z()*Hadamard_variable(noise)

def Cx(noise):
    final_matrix = np.zeros((2,2))
    Pi_0 = np.matrix([[1,0],[0,0]])
    final_matrix = sparse.kron(Pi_0,np.identity(2))
    Pi_1 = np.matrix([[0,0],[0,1]])
    final_matrix = sparse.kron(Pi_1,sigma_X_variable(noise))
    return final_matrix.A

### Sigle qubit quantum gate acting on the n-th qubit

In [38]:
def N_th_qubit_gate(quantum_gate,qubit_number):
    
    '''
    List below will hold gates acting on one qubit. For example, for L = 3,
    the Hadamard gate acting on the qubit 1 is given by = 1 x H x 1, where 
    x is the Kronecker product. Then, qubits_list = [1,H,1].

    ''' 
    qubits_list = [] 
    
    for i in range(L):
        
        if i == qubit_number: # qubit_number^th position in the list is the quantum_gate.
            
            qubits_list.append(quantum_gate)
            
        else: # Other gates are identity operators.
            
            qubits_list.append(np.identity(2))
            
    '''
    The following loop performs the Kronecker product.
    '''        
    
    final_matrix = sparse.csr_matrix(qubits_list[0]) # Initializes the final matrix.
    
    for g in range(1,len(qubits_list)):
        
        final_matrix = sparse.kron(qubits_list[g],final_matrix) # kronecker product.
        
    return final_matrix     

In [ ]:
def Grover_operator(Delta):
    
    '''
    First the U_x will be created.
    '''
    U_x = np.identity(2**L, dtype = complex);
    for i in range(L-1):
        np.matmul(U_x,N_th_qubit_gate(sigma_X_variable()))

In [31]:

U_x = np.identity(2**L, dtype = complex);